In [ ]:
%matplotlib inline

In [ ]:
%matplotlib notebook
import multiprocessing as mp
mp.set_start_method('forkserver')

In [ ]:
import torch.nn.functional as F
import torch
torch.cuda.set_device(1)
import torch.nn as nn
from torch.nn.utils import weight_norm, spectral_norm
from tqdm import tqdm_notebook as tqdm
from collections import Counter
import numpy as np
import numpy as np 
from matplotlib import pyplot as plt
from torch.optim import *
from InvertibleResnet import *

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [   transforms.Pad(2),
        transforms.ToTensor(),     
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

In [ ]:
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                         shuffle=True, num_workers=2)

In [ ]:
testnet = InvertibleResnetConv(1,32, list_num_blocks=(32,32,32)).cuda()

In [ ]:
b = testnet(next(iter(trainloader))[0].cuda(), return_logdet=True)

In [ ]:
from torch.distributions import *
std = 1
ndim = 32*32
height,width = 32,32
n_bins = 5
pzes = []
logdets = []
losses = []
latent_numpys =[]
gaussian = Normal(0.0,std)
normal_np = gaussian.sample((20000,2)).numpy()

In [ ]:
net = InvertibleResnetConv(1,32, list_num_blocks=(32,32,32)).cuda()
optim = Adam(net.parameters(), lr=0.001)

In [ ]:
latent = net(testset[0][0].unsqueeze(0).cuda(), return_logdet=False)
fixed_z = [torch.randn_like(l[:1]).cuda() for l in latent]
print ([l.shape for l in fixed_z])

In [ ]:
%matplotlib notebook
fig,(ax,ax2) = plt.subplots(1,2,figsize=(18,8))
axim = None
axscatter = None
fig.canvas.draw()

use_normal = True

def torchSigWMW( target,pred, beta=8):    
    mask = target.type(torch.cuda.ByteTensor)
    x = torch.masked_select(pred, 1+(-1)*mask).view(1,-1)
    y = torch.masked_select(pred, mask).view(-1,1)
    xn = x.expand(y.size(0), x.size(1))
    yn = y.expand(y.size(0), x.size(1))
    ur = torch.sigmoid(-beta*(xn-yn))
    return torch.sum(ur) / torch.sum(target) / torch.sum(1-target)

def criterion(pred, target):
    # return ((pred - (target * dist * 2 - dist)) ** 2).mean() 
    return torchSigWMW(target, pred)


for epoch in tqdm(range(0,200)):  
    itercount = 0
    for inp, _ in trainloader:
        
        net.train()
        optim.zero_grad()
        inp = inp.cuda()
        latent, logdet = net(inp + (torch.rand_like(inp).cuda() - 0.5) / ( n_bins)  , return_logdet=True, num_logdet_iter=1) 
        logdet /= inp.shape[2] * inp.shape[3]
        flat_latent = torch.cat([l.view(inp.shape[0], -1) for l in latent], dim=1)
        latentstd = torch.mean(flat_latent.std(dim=0))
        latentstdmax = torch.max(flat_latent.std(dim=0))
        latentstdmin = torch.min(flat_latent.std(dim=0))

        pz = -(flat_latent ** 2).sum(dim=1) / 2 / std /std            
        pz = pz.mean()
        pz /= inp.shape[2] * inp.shape[3]
        pz -= 0.5 * np.log(2 * 3.1415 * std * std)
        
        np_loss = ( np.log(n_bins) -logdet.item()-pz.item()) / np.log(2)
        print ('loss:{:.4f}, logdet:{:.4f} pz:{:.4f} latentstd:{:.4f} latentstdmax:{:.4f} latentstdmin:{:.4f}'.format(
                np_loss, logdet.item(),pz.item(),latentstd.item(),latentstdmax.item(),latentstdmin.item()))
        ( - logdet - pz).backward()
        if torch.isnan(logdet):
            raise Exception()
        pzes.append(pz.item())
        logdets.append(logdet.item())
        losses.append(np_loss)
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        optim.step()

        if itercount % 10 == 0:
            net.eval()
            latents = []
            with torch.no_grad():
                latent_numpy = []
                for test_batch, _ in testloader:
                    test_batch = test_batch.cuda()
                    latent = net(test_batch + (torch.rand_like(test_batch).cuda() - 0.5) / ( n_bins), return_logdet=False)
                    _latent_numpy = torch.cat([l.contiguous().view(test_batch.shape[0], -1) for l in latent], dim=1).detach().cpu().numpy()
                    latent_numpy.append(_latent_numpy)
                    break
                latent_numpy = np.concatenate(latent_numpy)

                pred = net([l[:1] for l in latent], reverse=True, reverse_iterations=200)                
                pred_sample = net(fixed_z, reverse=True, reverse_iterations=200)
            
            

            recon_loss = F.mse_loss(test_batch[:1], pred)
            if itercount >0:
                print ('loss:{:.4f}, logdet:{:.4f} pz:{:.4f} latentstd:{:.4f} latentmean:{:.4f} recon_loss:{:.4f}'.format(
                np_loss, logdet.item(),pz.item(), np.mean(latent_numpy.std(axis=0)),np.mean(latent_numpy.mean(axis=0)),  recon_loss.item()))



            pred_np = pred[0].cpu().detach().numpy().reshape((height,width))
            inp_np = test_batch[0].cpu().detach().numpy().reshape((height,width))            
            pred_sample_np = pred_sample[0].cpu().detach().numpy().reshape((height,width))
            disp = np.concatenate([pred_np, inp_np, pred_sample_np], axis=1)
            if axim:
                axim.set_data(disp)
            else:
                axim = ax.imshow(disp, vmin=-1.0, vmax=1.0)


            ax2.clear()
            ax2.scatter(normal_np[:1000,0], normal_np[:1000,1], c='g')            
            ax2.scatter(latent_numpy[:1000,-1], latent_numpy[:1000,-2],  c='r')  
            fig.canvas.draw()
            latent_numpys.append(latent_numpy) 
        itercount += 1
        del pz,  logdet, latent, latentstd
    #with torch.no_grad():
    #    for name, param in net.named_parameters():
    #        if '.weight' in name:
    #            param /= torch.sqrt((param ** 2).sum())

In [ ]:
latent_numpys = np.array(latent_numpys)

In [ ]:
dists = torch.from_numpy(np.array([[dist,-dist], [dist, dist], [-dist, dist], [-dist,-dist]]).astype(np.float32)).cuda()

In [ ]:
latent_numpys.shape

In [ ]:
latent_numpys = latent_numpys.reshape((-1,80,2560,2))

In [ ]:
latent_numpy = latent_numpys[0,-1]

In [ ]:
latent_numpy.shape

In [ ]:
np.std(latent_numpy[np.logical_and(latent_numpy[:,0] > 0, latent_numpy[:,1] < 0)], axis=0)

In [ ]:
np.mean(latent_numpy[np.logical_and(latent_numpy[:,0] < 0, latent_numpy[:,1] < 0)], axis=0)

In [ ]:
%matplotlib notebook
fig,ax =  plt.subplots(1,1,figsize=(8,8))
def scroll(event):
    global i, latent_numpys
    oldi = i
    if event.key == 'e':
        i -= 1
    elif event.key =='d':
        i += 1
    if i < 0: i = 0
    if i >= len(latent_numpys[0]) - 0: i = len(latent_numpys[0]) - 1
    if oldi != i:
        ax.clear()
        latent_numpy = latent_numpys[0][i][-1]
        
        ax.clear()
        _ = ax.hist(latent_numpys[0][i][:,1], bins=100, density=True)
        _ = ax.hist(normal_np[:,1], bins=100, density=True)
        
        fig.canvas.draw()
        #ax.set_xlim(-3,3)
        #ax.set_ylim(-3,3)


cid1 = fig.canvas.mpl_connect('key_press_event', scroll)
i = 0
latent_numpy = latent_numpys[0][i][-1]
ax.clear()
_ = ax.hist(latent_numpys[0][i][:,1], bins=100, density=True)
_ = ax.hist(normal_np[:,1], bins=100, density=True)


fig.canvas.draw()
#ax.set_xlim(-3,3)
#ax.set_ylim(-3,3)


In [ ]:
%matplotlib notebook
fig,ax =  plt.subplots(1,1,figsize=(8,8))
def scroll(event):
    global i, latent_numpys
    oldi = i
    if event.key == 'e':
        i -= 1
    elif event.key =='d':
        i += 1
    if i < 0: i = 0
    if i >= len(latent_numpys[0]) - 0: i = len(latent_numpys[0]) - 1
    if oldi != i:
        ax.clear()
        latent_numpy = latent_numpys[0,i]
        
        ax.clear()
        ax.scatter(normal_np[:,0], normal_np[:,1], c='g') 
        ax.scatter(latent_numpy[:500,0], latent_numpy[:500,1],  c='r')  
        fig.canvas.draw()
        #ax.set_xlim(-3,3)
        #ax.set_ylim(-3,3)


cid1 = fig.canvas.mpl_connect('key_press_event', scroll)
i = 0
latent_numpy = latent_numpys[0,i]
ax.clear()
ax.scatter(normal_np[:,0], normal_np[:,1], c='g')
ax.scatter(latent_numpy[:500,0], latent_numpy[:500,1],  c='r')  
fig.canvas.draw()
#ax.set_xlim(-3,3)
#ax.set_ylim(-3,3)

In [ ]:
%matplotlib inline
plt.plot(range(len(losses[:])), losses[:])

In [ ]:
%matplotlib inline
plt.plot(range(len(logdets[:])), logdets[:])

In [ ]:
%matplotlib inline
plt.plot(range(len(pzes[250:])), pzes[250:])